In [9]:
# compare phase vs. percent transition
# compare amplitude vs. percent transition

In [10]:
import sys
import rosbag
import matplotlib.pyplot as plt
import numpy as np

In [11]:
# obtain percent transition vs. time
def getWeights(filename):
    weights = []
    times = []
    bag = rosbag.Bag(filename)
    for topic, msg, t in bag.read_messages(topics=["/roboy_dep/linear_combination"]):
        weights.append(msg.weights)
        times.append(t.to_sec())
    bag.close()
    return np.array(weights), np.array(times)

In [12]:
filename = "/home/roboy/dep_data/data/combination/10FB04FS_04FB10FS_200s_2017-10-18-12-50-31.bag"

In [13]:
weights, times = getWeights(filename)

In [14]:
# obtain data from file
def getData(filename):
    position_to_rads = 2.0*3.14159/(2000.0*53.0);
    displacement_to_N = 0.237536
    time = []
    pos = []
    force = []
    bag = rosbag.Bag(filename)
    for topic, msg, t in bag.read_messages(topics=["/roboy/middleware/MotorStatus"]):
        pos.append(msg.position)
        force.append(msg.displacement)
        time.append(t.to_sec())
    bag.close()
    time = np.array(time)
    pos = np.array(pos)*position_to_rads
    force = np.array(force)*displacement_to_N
    return time, pos, force

In [15]:
time, pos, force = getData(filename)

In [16]:
# need to remove bit of recording before and after actual experiment
# inspect plot and provide approximate cutoff times
t_start = 4.3
t_end = 205.0

In [17]:
# obtain minima of given muscle data
from scipy.signal import argrelextrema
def getPeaks(position):
    indices = np.array(argrelextrema(position, np.less))
    return indices

ImportError: No module named scipy.signal

We can calculate an approximate phase of a given muscle relative to its counter part in a given behavior by: 
1. calculating the relative position of the muscle in comparison with an unchanging muscle through comparing time difference between peaks, 
2. calculating the original time difference between peaks for the muscle in the original behavior, and 
3. calculating a phase by comparing the total time difference relative to the period

##### Obtaining muscle and reference peak indices

In [ ]:
def movingAvg(data, window_size):
    a = []
    for i in range(len(data)):
        a.append(np.ma.average(data[np.clip(i-int(window_size/2),0,len(data)):np.clip(i+int(window_size/2),0,len(data))]))
    return np.array(a)

In [ ]:
# remove indices above/below cutoff times from peaks
def trim(indices,t_start,t_end):
    i_start = int(t_start/0.020)
    i_end = int(t_end/0.020)
    indices[indices < i_start] = 0
    indices[indices > i_end] = 0
    indices = np.trim_zeros(indices)
    return indices

In [ ]:
# For FB to FS
# change muscle = muscle 2 i.e. position[:,3]
# take moving average in attempt to differentiate two consecutive points with the same value
muscle = movingAvg(pos[:,3],3)
muscle_peaks = getPeaks(muscle)[0]
muscle_peaks = trim(muscle_peaks,t_start,t_end)
# reference muscle = muscle 5 i.e. position[:,5]
ref = movingAvg(pos[:,5],3)
ref_peaks = getPeaks(ref)[0]
ref_peaks = trim(ref_peaks,t_start,t_end)

# target muscle has 3 peaks after end of experiment -> remove these
#muscle_peaks = muscle_peaks[:-3]
# reference muscle has 4 peaks after end of experiment -> remove these
#ref_peaks = ref_peaks[:-2]


In [ ]:
print muscle_peaks, ref_peaks

In [ ]:
avg_dist = np.ma.average((ref_peaks-np.roll(ref_peaks,1))[1:])
std_dist = np.std((ref_peaks-np.roll(ref_peaks,1))[1:])
print avg_dist, std_dist

##### Behavior period and reference distance

In [ ]:
avg_period = (1358.5-717.0)/5
dist = 758.0 - 808.0

##### Phase plot

In [ ]:
y = ((muscle_peaks-ref_peaks))/avg_dist*360#-dist)#/avg_dist*360
y

In [ ]:
x = np.around(times[muscle_peaks]-times[0],2)

In [ ]:
plt.figure(1)
plt.plot(x,y)
plt.show()

In [1]:
a = np.array(range(11))

NameError: name 'np' is not defined